In [1]:
# from google.colab import userdata
# tok = userdata.get('git_token')

In [1]:
# !pip install torchmetrics -Uqq
from torchmetrics.classification import BinaryAccuracy

/home/dai/anaconda3/envs/pytorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import torch
from torch import nn
from torch.nn import functional as F
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [3]:
# df=pd.read_csv("train.csv")
# y=df["smoking"]
# df=df.drop(["id", "smoking"],axis=1)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
import torch.nn as nn

class ClsModel(nn.Module):
    def __init__(self, input_size, hidden_size, out_size, dropout_rate=0.3):
        super().__init__()
        self.model = nn.Sequential(
          nn.Linear(in_features=input_size, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=20, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=20, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.3),
          nn.Linear(in_features=hidden_size, out_features=hidden_size, dtype=torch.float64),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(in_features=hidden_size, out_features=out_size, dtype=torch.float64),
          nn.Sigmoid()
      )

    def forward(self, data):
        return self.model(data)


In [6]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# df = scaler.fit_transform(df)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x_sample = self.X[idx]
        y_sample = self.y.iloc[idx]
        return x_sample.astype("float64"), y_sample.astype("float64")

In [8]:
df=pd.read_csv("train.csv")
y=df["smoking"]
df=df.drop(["id", "smoking"],axis=1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = scaler.fit_transform(df)

def get_data(X,y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True,stratify=y)
    train_dataset = CustomDataset(X,y)
    val_dataset = CustomDataset(X_test,y_test)
    train_loader = DataLoader(train_dataset,batch_size=128,num_workers=os.cpu_count(),pin_memory=True)
    val_loader = DataLoader(val_dataset,batch_size=32,num_workers=os.cpu_count(),pin_memory=True)
    return train_loader, val_loader

In [9]:
train_loader, val_loader = get_data(df,y)

In [10]:
def save_checkpoint(model, optimizer, epoch, loss, filename):
    path = filename + '.pth'
    torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
      'loss': loss,
  }, path)

def load_checkpoint(model, optimizer, filename):
    path = filename + '.pth'
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    model.eval()
    return model, optimizer, epoch, loss

In [11]:
torch.manual_seed(42)
model = ClsModel(22,10,1)
model.to(device)
# model.compile()
loss_fn = nn.BCELoss()
adam = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler = ReduceLROnPlateau(adam, 'min')

In [12]:
# model , adam, epoch, loss = load_checkpoint(model, adam, 'model')
# adam = torch.optim.Adam(model.parameters(),lr=0.01)

In [16]:
for i in range(50):
    #Training
    model.train()
    accumulated_loss = 0
    print(f"Epoch: {i}")
    for idx,batch in enumerate(train_loader):
        msg = f"batch: {idx}"
        print('\r' + msg, end='', flush=True)
        X,y = batch
        X = X.to(device)
        y = y.to(device)
        preds = model(X)
        loss = loss_fn(preds,y.reshape(-1,1))
        adam.zero_grad()
        loss.backward()
        adam.step()
        accumulated_loss += loss.item()
    print()
    _loss = accumulated_loss/len(train_loader)
    scheduler.step(_loss)
    print(accumulated_loss/len(train_loader))
    save_checkpoint(model, adam, i, accumulated_loss/len(train_loader), '_model')

Epoch: 0


batch: 1244
0.47478213210384146
Epoch: 1
batch: 1244
0.4744336729270749
Epoch: 2
batch: 1244
0.4738583168650077
Epoch: 3
batch: 1244
0.47417310528717005
Epoch: 4
batch: 1244
0.47459841639251993
Epoch: 5
batch: 1244
0.4743288874094076
Epoch: 6
batch: 1244
0.4739260733227308
Epoch: 7
batch: 1244
0.4728679226135789
Epoch: 8
batch: 1244
0.4747306203662553
Epoch: 9
batch: 1244
0.4735261027431214
Epoch: 10
batch: 1244
0.47386232788541216
Epoch: 11
batch: 1244
0.47497985247384733
Epoch: 12
batch: 1244
0.4736299347612075
Epoch: 13
batch: 1244
0.473081113281483
Epoch: 14
batch: 1244
0.47504151261278404
Epoch: 15
batch: 1244
0.47419123510598127
Epoch: 16
batch: 1244
0.47499526267440645
Epoch: 17
batch: 1244
0.47310094722534285
Epoch: 18
batch: 1244
0.4741920298498847
Epoch: 19
batch: 1244
0.47265155796730823
Epoch: 20
batch: 1244
0.4718707261634628
Epoch: 21
batch: 1244
0.47203909194956145
Epoch: 22
batch: 1244
0.4707405427053348
Epoch: 23
batch: 1244
0.4712050274392913
Epoch: 24
batch: 1244
0.4

In [14]:
# os.cpu_count()

In [ ]:
# df=pd.read_csv("/content/train.csv")
# y=df["smoking"]
# df=df.drop(["id", "smoking"],axis=1)
# X_train,X_test,y_train,y_test = train_test_split(df,y,random_state=42,shuffle=True,stratify=y)

In [ ]:
# test_X = torch.Tensor(X_test.to_numpy()).to(torch.float64)
# test_y = torch.Tensor(y_test.to_numpy()).to(torch.float64)

In [ ]:
# metrics = BinaryAccuracy().to(device)

In [ ]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [ ]:
save_checkpoint(model,adam,40,0.457, "model3")

In [ ]:
test = pd.read_csv("test.csv",index_col="id")
test = scaler.transform(test)
test = torch.Tensor(test).to(torch.float64)

In [ ]:
# with torch.inference_mode():
#     preds = model(test_X.to(device))
#     acc = metrics(preds,test_y.reshape(-1,1).to(device))
#     print(acc)

In [ ]:
with torch.inference_mode():
    preds = model(test.to(device))
    print(preds)

tensor([[0.5624],
        [0.4480],
        [0.4027],
        ...,
        [0.5584],
        [0.1953],
        [0.1139]], dtype=torch.float64)


In [ ]:
submission = pd.read_csv("sample_submission.csv", index_col="id")
submission['smoking'] = preds.cpu()

In [ ]:
submission.to_csv("submission.csv")